In [ ]:
!pip install torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 6.7 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install '/content/drive/MyDrive/AI/project3/dataset/en_core_web_md-3.2.0.tar.gz'
!pip install '/content/drive/MyDrive/AI/project3/dataset/de_core_news_md-3.2.0.tar.gz'

Processing ./drive/MyDrive/AI/project3/dataset/en_core_web_md-3.2.0.tar.gz
     |████████████████████████████████| 6.0 MB 8.8 MB/s 
     |████████████████████████████████| 628 kB 49.2 MB/s 
     |████████████████████████████████| 181 kB 48.5 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 451 kB 49.8 MB/s 
     |████████████████████████████████| 10.1 MB 57.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-3.2.0-py3-none-any.whl size=45684465 sha256=ca12fb9a19fb811c9eafddf10a22662333084f7c1bfd8b43dc9422110efa8acb
  Stored in directory: /root/.cache/pip/wheels/cc/79/d2/b7ad871b150b1942ca4fe921c603e71c225e780276a69c6331
Successfully built en-core-web-md
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:


In [ ]:
import math
import time
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import io
import re
import spacy

from collections import Counter
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab, FastText
from torchtext.data.metrics import bleu_score
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

from torch import Tensor
from torch import zeros
from typing import Tuple

random.seed(20)
torch.manual_seed(20)

In [ ]:
nlp1 = spacy.load('en_core_web_md')
nlp2 = spacy.load('de_core_news_md')

测试集预处理，将.sgm格式的文件中的句子内容抽取出来，存入.txt文件

In [ ]:
eng_test = []
with open("/content/drive/MyDrive/AI/project3/dataset/newstest2016-ende-src.en.sgm", "r") as f1:
  data3 = f1.read().split("\n")
  for i in range(len(data3)):
    temp = re.findall(r'<seg.*?>(.*?)</seg>',data3[i],re.S|re.M)
    if not temp:
      continue
    else:
      eng_test.append(temp[0])
f1.close()

with open("/content/drive/MyDrive/AI/project3/dataset/en_test.txt", "w") as f2:
  for line in eng_test:
    f2.write(line+"\n")
f2.close()

In [ ]:
de_test = []
with open("/content/drive/MyDrive/AI/project3/dataset/newstest2016-ende-ref.de.sgm", "r") as f3:
  data4 = f3.read().split("\n")
  for i in range(len(data4)):
    temp = re.findall(r'<seg.*?>(.*?)</seg>',data4[i],re.S|re.M)
    if not temp:
      continue
    else:
      de_test.append(temp[0])
f3.close()

with open("/content/drive/MyDrive/AI/project3/dataset/de_test.txt", "w") as f4:
  for line in de_test:
    f4.write(line+"\n")
f4.close()

In [ ]:
en_train = []
with open("/content/drive/MyDrive/AI/project3/dataset/news-commentary-v11.de-en.en", "r") as f5:
  data4 = f5.read().split("\n")
  en_train = data4[0:10000]

f5.close()

with open("/content/drive/MyDrive/AI/project3/dataset/en_train.txt", "w") as f6:
  for line in en_train:
    f6.write(line+"\n")
f6.close()

In [ ]:
de_train = []
with open("/content/drive/MyDrive/AI/project3/dataset/news-commentary-v11.de-en.de", "r") as f7:
  data4 = f7.read().split("\n")
  de_train = data4[0:10000]

f7.close()

with open("/content/drive/MyDrive/AI/project3/dataset/de_train.txt", "w") as f8:
  for line in de_train:
    f8.write(line+"\n")
f8.close()

In [ ]:
class DataPipeline:

    def __init__(self, batch_size=64):
        train_datasets = ["/content/drive/MyDrive/AI/project3/dataset/en_train.txt", "/content/drive/MyDrive/AI/project3/dataset/de_train.txt"]
        val_datasets = ["/content/drive/MyDrive/AI/project3/dataset/newstest2010.en", "/content/drive/MyDrive/AI/project3/dataset/newstest2010.de"]
        test_datasets = ["/content/drive/MyDrive/AI/project3/dataset/en_test.txt", "/content/drive/MyDrive/AI/project3/dataset/de_test.txt"]
        self.en_tokenizer = get_tokenizer('spacy', language='en_core_web_md')
        self.de_tokenizer = get_tokenizer('spacy', language='de_core_news_md')
        self.en_vocab = self.build_en_vocab(train_datasets[0], self.en_tokenizer)
        self.de_vocab = self.build_de_vocab(train_datasets[1], self.de_tokenizer)

        train_data = self.create_tensor(train_datasets)
        val_data = self.create_tensor(val_datasets)
        test_data = self.create_tensor(test_datasets)

        self.train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=self.create_batch)
        self.valid_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, collate_fn=self.create_batch)
        self.test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, collate_fn=self.create_batch)

    def build_en_vocab(self, filepath, tokenizer):
        counter = Counter()
        with io.open(filepath, encoding="utf8") as f1:
            for temp in f1:
                counter.update(tokenizer(temp))
        vocab = Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'], vectors=FastText(language='en', max_vectors=1000_000))
        zero_vec = torch.zeros(vocab.vectors.size()[0])
        zero_vec = torch.unsqueeze(zero_vec, dim=1)
        vocab.vectors = torch.cat((zero_vec, zero_vec, zero_vec, vocab.vectors), dim=1)
        vocab.vectors[vocab["<pad>"]][0] = 1
        vocab.vectors[vocab["<bos>"]][1] = 1
        vocab.vectors[vocab["<eos>"]][2] = 1
        return vocab

    def build_de_vocab(self, filepath, tokenizer):
        counter = Counter()
        with io.open(filepath, encoding="utf8") as f:
            for temp in f:
                counter.update(tokenizer(temp))
        return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

    def create_batch(self, data_batch):
        en_batch, de_batch, = [], []
        for (en_item, de_item) in data_batch:
            en_batch.append(torch.cat([torch.tensor([self.en_vocab['<bos>']]), en_item, torch.tensor([self.en_vocab['<eos>']])], dim=0))
            de_batch.append(torch.cat([torch.tensor([self.de_vocab['<bos>']]), de_item, torch.tensor([self.de_vocab['<eos>']])], dim=0))
        en_batch = pad_sequence(en_batch, padding_value=self.en_vocab['<pad>'])
        de_batch = pad_sequence(de_batch, padding_value=self.de_vocab['<pad>'])
        return en_batch, de_batch

    def create_tensor(self, filepaths):
        raw_en_iter = iter(io.open(filepaths[0], encoding="utf8"))
        raw_de_iter = iter(io.open(filepaths[1], encoding="utf8"))
        data = []
        #cnt = 0
        for (raw_en, raw_de) in zip(raw_en_iter, raw_de_iter):
            #cnt += 1
            en_tensor_ = torch.tensor([self.en_vocab[token] for token in self.en_tokenizer(raw_en)[:-1]], dtype=torch.long)  
            de_tensor_ = torch.tensor([self.de_vocab[token] for token in self.de_tokenizer(raw_de)[:-1]], dtype=torch.long)  
            data.append((en_tensor_, de_tensor_))
            #if cnt >= 20000:
              #break
        return data


In [ ]:
class Encoder(nn.Module):
    def __init__(self, dim_input: int, dim_en_embedding: int, dim_en_hidden: int, dim_de_hidden: int, dropout: float):
        super().__init__()

        self.dim_input = dim_input
        self.dim_en_embedding = dim_en_embedding
        self.dim_en_hidden = dim_en_hidden
        self.dim_de_hidden = dim_de_hidden
        self.dropout = dropout

        self.embedding = nn.Embedding(dim_input, dim_en_embedding).from_pretrained(en_vocab.vectors, freeze=True)
        self.rnn = nn.GRU(dim_en_embedding, dim_en_hidden, num_layers=1, bidirectional=True)
        self.fc = nn.Linear(dim_en_hidden * 2, dim_de_hidden)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src: Tensor) -> Tuple[Tensor]:
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)))
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim_en_hidden: int, dim_de_hidden: int, dim_attention: int):
        super().__init__()

        self.dim_en_hidden = dim_en_hidden
        self.dim_de_hidden = dim_de_hidden
        self.attn_in = (dim_en_hidden * 2) + dim_de_hidden
        self.attn = nn.Linear(self.attn_in, dim_attention)

    def forward(self, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tensor:
        src_len = encoder_outputs.shape[0]
        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((repeated_decoder_hidden, encoder_outputs), dim=2)))

        attention = torch.sum(energy, dim=2)
        return F.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, dim_output: int, dim_de_embedding: int, dim_en_hidden: int, dim_de_hidden: int, dropout: float, attention: nn.Module):
        super().__init__()

        self.dim_de_embedding = dim_de_embedding
        self.dim_en_hidden = dim_en_hidden
        self.dim_de_hidden = dim_de_hidden
        self.dim_output = dim_output
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(dim_output, dim_de_embedding)
        self.rnn = nn.GRU((dim_en_hidden * 2) + dim_de_embedding, dim_de_hidden)
        self.out = nn.Linear(self.attention.attn_in + dim_de_embedding, dim_output)
        self.dropout = nn.Dropout(dropout)

    def _weighted_encoder_rep(self, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tensor:
        a = self.attention(decoder_hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)
        return weighted_encoder_rep

    def forward(self, input: Tensor, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tuple[Tensor]:
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden, encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim=2)
        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output, weighted_encoder_rep, embedded), dim=1))

        return output, decoder_hidden.squeeze(0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder: nn.Module, decoder: nn.Module, device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src: Tensor, trg: Tensor, teacher_forcing_ratio: float = 0.5) -> Tensor:
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.dim_output

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <bos> token
        output = trg[0, :]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

In [ ]:
data = DataPipeline(batch_size=16)
en_vocab = data.en_vocab
de_vocab = data.de_vocab
train_loader = data.train_loader
valid_loader = data.valid_loader
test_loader = data.test_loader

dim_input = len(en_vocab)
dim_output = len(de_vocab)
dim_en_embedding = en_vocab.vectors.size()[1]
dim_de_embedding = 64
dim_en_hidden = 128
dim_de_hidden = 128
dim_attention = 32
dropout_en = 0.5
dropout_de = 0.5

enc = Encoder(dim_input, dim_en_embedding, dim_en_hidden, dim_de_hidden, dropout_en)
attn = Attention(dim_en_hidden, dim_de_hidden, dim_attention)
dec = Decoder(dim_output, dim_de_embedding, dim_en_hidden, dim_de_hidden, dropout_de, attn)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=de_vocab.stoi['<pad>'])

In [ ]:
def train(model: nn.Module, iterator: torch.utils.data.DataLoader, optimizer: optim.Optimizer, criterion: nn.Module, clip: float):
    model.train()
    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model: nn.Module, iterator: torch.utils.data.DataLoader, criterion: nn.Module):
    model.eval()
    epoch_loss, epoch_bleu = 0, 0
    with torch.no_grad():
        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, 0)  # turn off teacher forcing
            epoch_bleu += bleu(output[1:, :, :], trg[1:, :], de_vocab, device)
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator), epoch_bleu / len(iterator)

In [ ]:
def calculate_time(start_time: float, end_time: float):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def bleu(output, target, vocab, device):
    _, output_indices = torch.max(output, 2)
    eos_pad = torch.full((1, output.size()[1]), vocab.stoi["<eos>"], device=device)
    output_indices = torch.cat((output_indices, eos_pad), 0)
    output_indices = output_indices.transpose(1, 0)
    target = target.transpose(1, 0)

    output_str = [[vocab.itos[x] for x in y] for y in output_indices]
    output_str = [x[:x.index("<eos>")] for x in output_str]
    target_str = [[vocab.itos[x] for x in y] for y in target]
    target_str = [[x[:x.index("<eos>")]] for x in target_str]
    return bleu_score(output_str, target_str)

In [ ]:
N_EPOCHS = 20
CLIP = 1

print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss, valid_bleu = evaluate(model, valid_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = calculate_time(start_time, end_time)

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\tVal. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f} | Val. Bleu: {round(100*valid_bleu, 2)}')

test_loss, test_bleu = evaluate(model, test_loader, criterion)



The model has 15,110,013 trainable parameters
Epoch: 01 | Time: 2m 10s
	Train Loss: 7.251 | Train PPL: 1409.779
	Val. Loss: 7.599 |  Val. PPL: 1996.950 | Val. Bleu: 0.0
Epoch: 02 | Time: 2m 12s
	Train Loss: 6.629 | Train PPL: 756.478
	Val. Loss: 7.623 |  Val. PPL: 2044.602 | Val. Bleu: 0.05
Epoch: 03 | Time: 2m 13s
	Train Loss: 6.144 | Train PPL: 465.779
	Val. Loss: 7.713 |  Val. PPL: 2236.797 | Val. Bleu: 0.07
Epoch: 04 | Time: 2m 12s
	Train Loss: 5.706 | Train PPL: 300.748
	Val. Loss: 7.881 |  Val. PPL: 2647.570 | Val. Bleu: 0.06
Epoch: 05 | Time: 2m 12s
	Train Loss: 5.299 | Train PPL: 200.039
	Val. Loss: 7.982 |  Val. PPL: 2926.870 | Val. Bleu: 0.1
Epoch: 06 | Time: 2m 12s
	Train Loss: 4.945 | Train PPL: 140.478
	Val. Loss: 8.123 |  Val. PPL: 3372.609 | Val. Bleu: 0.23
Epoch: 07 | Time: 2m 13s
	Train Loss: 4.649 | Train PPL: 104.526
	Val. Loss: 8.287 |  Val. PPL: 3970.586 | Val. Bleu: 0.19
Epoch: 08 | Time: 2m 12s
	Train Loss: 4.410 | Train PPL:  82.256
	Val. Loss: 8.433 |  Val. PPL

In [ ]:
print(f'\tTest Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} | Test Bleu: {round(100*test_bleu, 2)}')

	Test Loss: 9.215 | Test PPL: 10042.775 | Test Bleu: 0.41


In [ ]:
torch.save(model, '/content/drive/MyDrive/AI/project3/model1.pkl')